This function computes closed null geodesics of $ 2S(\mathbf{x}, t)-\mathcal{T}_0 \mathbf{I} $, where $ S(\mathbf{x}, t) = \begin{pmatrix} S^{11} && S^{12} \\ S^{12} && S^{22} \end{pmatrix} $ is the (symmetric) rate of strain tensor at $ (\mathbf{x}, t) $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| T0 | float | $ \mathcal{T}_0 $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| interp_phi_prime_denom | interpolant | $ \sin(2\phi)[S^{22}(\mathbf{x})-S^{11}(\mathbf{x})]+2\cos(2\phi)S^{12}(\mathbf{x}) $ | d_threshold | float | threshold distance between start of closed null-geodesic and maximum allowed first return distance to starting point |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}, \phi $) |
| x0T0 | list | list containing x-coordinates of initial conditions $ x_0(\mathcal{T}_0) $ |
| y0T0 | list | list containing y-coordinates of initial conditions $ y_0(\mathcal{T}_0) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# function computing initial conditions (depending on T_0)
from ipynb.fs.defs.init_level_set import _init_level_set

# find closed curve
from ipynb.fs.defs.closed_curve import closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

# tqdm shows progress bar
from tqdm.notebook import tqdm

# import parallel computing package
from joblib import Parallel, delayed

# library for Polygon, Point objects
from shapely.geometry import Polygon, Point

# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [ ]:
def closed_null_geodesics(X, Y, T0, interp_phi_prime, interp_phi_prime_denom, d_threshold, S11, verbose = False):
    '''
    Compute closed null geodesics. The algorithm returns closed null geodesics coinciding with elliptic LCS.
    
    Parameters:
        X:                      array (Ny, Nx),  X-meshgrid.
        Y:                      array (Ny, Nx),  Y-meshgrid.
        T0:                     float, T0 --> transport density used to compute ICs of null geodesics.
        interp_phi_prime:       Interpolant for phi_prime.
        interp_phi_prime_denom: Interpolant for phi_prime_denom (=domain of existence DOE).
        d_threshold:            float, threshold value for re-intersection of closed curve
        S11:                    array (Ny, Nx), S_{11}.
        verbose:                bool, if True, function reports progress at every 100th iteration.          
        
    Returns:
        solODEcurves:           list, solutions of ODE = set of closed null geodesics.
        x0T0:                   list, x-coordinates of ICs.
        y0T0:                   list, y-coordinates of ICs.
    '''
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(S11).astype(int)
    
    # compute initial conditions
    x0T0, y0T0, phi0T0 = _init_level_set(X, Y, S11, T0)
    
    # define integration domain of dummy variable.
    # Note if you do not find any structures, then I recommend increasing this value. 
    # For the majority of examples the algorithm converges much sooner. 
    # An upper bound on the dummy time-interval is included in order to avoid long computation times.
    s = [0, 12]
    
    # define resolution of trajectories
    # This resolution can also be refined.
    s_eval = np.linspace(s[0], s[-1], 4000)
    
    ds = s_eval[1]-s_eval[0]
    
    # number of initial conditions
    len_x0T0 = len(x0T0)
    
    solODE = np.zeros((len(s_eval), 3, len_x0T0))
    
    # initial conditions
    solODE[0, :, :] = np.array([x0T0, y0T0, phi0T0])
    
    for i in range(len(s_eval)-1):
        
        solODE[i+1, :, :] = RK4_integration(solODE[i, :, :], ds, interp_phi_prime)
    
    solODE_closed_curves = []
    
    print("Computation of closed null geodesics: Started...")
    
    # iterate over all initial conditions [x0T0, y0T0, phi0T0]
    for j in range(len_x0T0):
        
        # store x, y, phi
        # we do not solve the ODE for all ICs as this would require long computation times.
        # for higher accuracy just replace the subsequente three lines of codes with the commented version
        x = solODE[::2,0, j] #solODE[:,0,j]
        y = solODE[::2,1, j] #solODE[:,1,j]
        phi = solODE[::2,2, j] #solODE[:,2,j]
    
        if verbose:
            if j%100==0:
                print("Percentage completed: ", np.around(np.around(j/len_x0T0, 4)*100,2))
    
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed, y_closed, phi_closed = closed_curve(x, y, phi, d_threshold, X, Y, defined_domain)
        
        # store solutions
        solODE_closed_curves.append([x_closed, y_closed, phi_closed])
    
    print("Computation of closed null geodesics: Done!")
    
    return solODE_closed_curves, [x0T0, y0T0]